In [1]:
from datetime import datetime
from os import getcwd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from UliPlot.XLSX import auto_adjust_xlsx_column_width

In [2]:
cooper = pd.read_excel('cooper.xls')
ernest = pd.read_excel('ernest.xls')
aalto = pd.read_excel('aalto.xls')
leo = pd.read_excel('leo.xls')
howard = pd.read_excel('howard.xls')

jason = pd.read_excel('jason.xls')
chris = pd.read_excel('chris.xls')
wendy = pd.read_excel('wendy.xls')
evelyn = pd.read_excel('evelyn.xls')

In [3]:
pmsystem_cft = (pd.concat([cooper,ernest,aalto,leo,howard,jason,chris,wendy,evelyn])
.iloc[:,[0,44]]
.set_index('Project ID')
)

In [64]:
# cft_list_file = '(Input) CFTListExport2.xls'
# pmsystem_cft = pd.read_excel(cft_list_file, usecols=[0,44], index_col=[0])

In [4]:
pmsystem_project_list_file = '(Input) ProjectListExport.xls'
prweb_file = '(Input) New Production Status Summary.xlsx'

start_from = '2023-06'
consumer_dpqe_list = ['Lo, Aalto','Kuo, Chris','Wang, Cooper', 'Fu, Ernest','Hsu, Jason(PQE)','Ni, Wendy','Wang, Evelyn','Hsu, Jason(PQE);\nWang, Evelyn','Chen, Leo(PQE)','Jen, Howard'] 
# 'Hsu, Jason(PQE);\nWang, Evelyn' is for GPSMAP 9000 series

# PM system

Read

In [176]:
pmsystem_project = pd.read_excel(pmsystem_project_list_file, usecols=[0,1,3,4,5,6,9,10,12], parse_dates=[2,3,4,5], index_col=[0], engine='openpyxl')
# pmsystem_cft = pd.read_excel(cft_list_file, usecols=[0,44], index_col=[0])

Manually input MP date (2023/08)

In [185]:
product_mpdate_dict = {'Instinct Y' : '2024-03-04',
'Forerunner 270' : '2024-12-02',
'Approach Z84' : '2025-01-20',
'LiDAR Lite v5' : '2026-01-01',
'Fusion - 80 Series Marine Stereos' : '2025-10-25',
'DEZL Sideview' : '2024-11-4',
'Tread 6" plus optional handlebar controller' : '2024-08-19',
'2" Thin Watch' : '2024/11/18',
'Forerunner 970' : '2024/12/30',
'Dive Light with Sonar': '2024/6/30',
'HRM6-Advanced' : '2024-08-14',
'Approach S50' : '2025-01-01',
'HRM6-Basic' : '2025-01-13',
'Edge MTB' : '2025-02-18'
}
for project,mpdate in product_mpdate_dict.items():    
    if pmsystem_project.loc[pmsystem_project['Project Name'] == project,'MP Date'].isnull().values:
        pmsystem_project.loc[pmsystem_project['Project Name'] == project,'MP Date'] = mpdate
    else:
        pass
# Test code
# pmsystem_project.loc[pmsystem_project['Project Name'] == 'Edge MTB','MP Date']

Other handling

In [192]:
pmsystem_project.drop([14704], inplace=True)

In [193]:
pmsystem_project = pmsystem_project.query("~`Project Name`.str.contains('BMW')")

In [194]:
pmsystem_project = pmsystem_project[(pmsystem_project['PDP/Form107'] == 'PDP')]

In [195]:
pmsystem_project.head()

,Project Name,Working Prototype Built and Tested,Final Factory Prototypes Built,MP Date,PR Date,Global/Asia,PDP/Form107,Status
Project ID,,,,,,,,
16544,2024 Value Dash Cams,2023-06-30,2023-10-23,2024-03-18,2024-01-12,Global,PDP,Development
16319,Aviation Headset,2024-03-27,2024-09-25,2025-04-21,2025-01-24,Global,PDP,Development
15624,Catalyst Kart/Auto,2022-06-24,2023-02-24,2023-11-13,2023-08-31,Global,PDP,Development
16199,DC-2023 Premium Dash Cameras,2023-02-02,2023-08-18,2023-12-04,2023-10-13,Global,PDP,Development
16718,DEZL 2024,2023-09-08,2024-01-16,2024-06-10,2024-04-09,Global,PDP,Development


Merge project and CFT lists

In [196]:
pmsystem_project_cft = pd.merge(left=pmsystem_project, right=pmsystem_cft, left_index=True, right_index=True)
pmsystem_project_cft.sort_index().head()

,Project Name,Working Prototype Built and Tested,Final Factory Prototypes Built,MP Date,PR Date,Global/Asia,PDP/Form107,Status,Develop Product QE (DPQE)
Project ID,,,,,,,,,
2854,Product - GDC 7500,2013-04-30,NaT,2019-07-01,2018-10-12,Global,PDP,Development,"Kuo, Chris"
5711,Product - GLA 85,2014-10-30,NaT,2018-12-07,2016-08-31,Global,PDP,Development,"Kuo, Chris"
7563,Mercury,NaT,NaT,2022-01-17,2021-04-23,Global,PDP,Development,"Fu, Ernest"
7899,Product - GI 275,2018-04-04,NaT,2019-04-02,2019-03-22,Global,PDP,Development,"Kuo, Chris"
8510,Xero L60i Laser Navigator,2022-10-19,2023-07-17,2024-01-08,2023-10-16,Global,PDP,Development,"Kuo, Chris"


In [197]:
# (pmsystem_project_cft
#  .rename(columns={'Develop  Product QE (DPQE)':'DPQE'})
#  .query("DPQE.str.contains(';')")
# )

In [198]:
# (pmsystem_project_cft
#  .rename(columns={'Develop  Product QE (DPQE)':'DPQE'})
#  .query("DPQE == 'Fu, Ernest'")
# )

Find projects which WW/ASIA SKU have different DPQE

In [199]:
ww_asia_diff = pmsystem_project_cft.copy()

In [200]:
#Handle Asis words in 'Project Name' column
ww_asia_diff['Project Name'] = ww_asia_diff['Project Name'].apply(lambda name : name.split(',')[0])
ww_asia_diff['Project Name'] = ww_asia_diff['Project Name'].str.replace('ASIA','')
ww_asia_diff['Project Name'] = ww_asia_diff['Project Name'].str.replace('Asia','')

In [201]:
# check any missing
ww_asia_diff[ww_asia_diff['Project Name'].str.contains('ASIA|asia|Asia')]

,Project Name,Working Prototype Built and Tested,Final Factory Prototypes Built,MP Date,PR Date,Global/Asia,PDP/Form107,Status,Develop Product QE (DPQE)
Project ID,,,,,,,,,


In [202]:
ww_asia_diff = ww_asia_diff[['Project Name','Develop  Product QE (DPQE)','Global/Asia','MP Date']].loc[ww_asia_diff['MP Date']> start_from]

In [203]:
# Find duplicated projects
ww_asia_diff = ww_asia_diff[ww_asia_diff.duplicated(subset=['Project Name'],keep=False)]
ww_asia_diff

,Project Name,Develop Product QE (DPQE),Global/Asia,MP Date
Project ID,,,,
9946,GPSMAP 9000,"Hsu, Jason(PQE);\nWang, Evelyn",Global,2023-08-07
9946,GPSMAP 9000,"Hsu, Jason(PQE);\nWang, Evelyn",Global,2023-08-07
10901,eTrex Solar,"Kuo, Chris",Global,2023-10-11
10926,Approach R50,"Kuo, Chris",Global,2024-03-04
12799,Descent S1 Buoy,"Lo, Aalto",Global,2024-03-11
12994,Descent Mk3 51mm,"Wang, Cooper",Global,2023-10-23
14069,Descent S1 Buoy,"Lo, Aalto",Asia,2024-03-11
15509,Approach R50,"Kuo, Chris",Asia,2024-03-04
15519,Venu 3S,"Chen, Leo(PQE)",Global,2023-07-03


In [204]:
# Find projects have different DPQE
ww_asia_diff_DPQE = ww_asia_diff[~ww_asia_diff.duplicated(subset=['Project Name','Develop  Product QE (DPQE)'],keep=False)]

Find projects w/o DPQE (include ASIA SKUs)

In [205]:
pmsystem_project_cft[pmsystem_project_cft['Develop  Product QE (DPQE)'].isnull()].dropna(thresh=5, axis=0)

,Project Name,Working Prototype Built and Tested,Final Factory Prototypes Built,MP Date,PR Date,Global/Asia,PDP/Form107,Status,Develop Product QE (DPQE)
Project ID,,,,,,,,,


In [206]:
projects_wo_DPQE = pmsystem_project_cft[pmsystem_project_cft['Develop  Product QE (DPQE)'].isnull()].dropna(thresh=5, axis=0).query("`MP Date` > @start_from")

Find projects with multi-DPQE (include ASIA SKUs)

In [207]:
multi_dpqe = pmsystem_project_cft[['Project Name','Global/Asia','Develop  Product QE (DPQE)','MP Date']]
multi_dpqe = multi_dpqe[multi_dpqe['MP Date']>start_from]
multi_dpqe.dropna(inplace=True)

In [208]:
multi_dpqe = multi_dpqe[multi_dpqe['Develop  Product QE (DPQE)'].str.contains('\n')]

Find projects w/o any date

In [209]:
# Find all date is NaT
# 1. Select dataframe w/ date (pmsystem_project_cft.iloc[:,1:5])
# 2. Rows w/ any date marked as True (notnull().any(axis=1))
# 3. Reverse (~)
pmsystem_project_wo_any_date = pmsystem_project_cft.loc[(~pmsystem_project_cft.iloc[:,1:5].notnull().any(axis=1))].reset_index().sort_values(['Develop  Product QE (DPQE)','Project Name']).set_index('Develop  Product QE (DPQE)')
pmsystem_project_wo_any_date

,Project ID,Project Name,Working Prototype Built and Tested,Final Factory Prototypes Built,MP Date,PR Date,Global/Asia,PDP/Form107,Status
Develop Product QE (DPQE),,,,,,,,,
"Fu, Ernest",16414,"Lily 2, Asia",NaT,NaT,NaT,NaT,Asia,PDP,Development
"Fu, Ernest",10692,SaMD Sleep Apnea Risk Assessment,NaT,NaT,NaT,NaT,Global,PDP,Concepts
"Hsu, Jason(PQE)",8752,Marine inReach Antenna,NaT,NaT,NaT,NaT,Global,PDP,Concepts
"Hsu, Jason(PQE)",16501,Panoptix PS22-IF Transducer and Ice Fishing Bu...,NaT,NaT,NaT,NaT,Global,PDP,Concepts
"Jen, Howard",16935,EPIX (Gen 3) Connected,NaT,NaT,NaT,NaT,Global,PDP,Concepts
"Kuo, Chris",13036,Alpha Max,NaT,NaT,NaT,NaT,Global,PDP,Concepts
"Kuo, Chris",13034,Companion Dog Tracking Collar,NaT,NaT,NaT,NaT,Global,PDP,Concepts
"Kuo, Chris",14604,L20 Laser Range Finder,NaT,NaT,NaT,NaT,Global,PDP,Concepts
"Kuo, Chris",14739,inReach Certus IoT,NaT,NaT,NaT,NaT,Global,PDP,Concepts


Melt

In [210]:
df_consumer_dpqe = pmsystem_project_cft[pmsystem_project_cft['Develop  Product QE (DPQE)'].isin(consumer_dpqe_list)].sort_index()
df_consumer_dpqe = df_consumer_dpqe[df_consumer_dpqe['Global/Asia']=='Global']
df_consumer_dpqe.head()

,Project Name,Working Prototype Built and Tested,Final Factory Prototypes Built,MP Date,PR Date,Global/Asia,PDP/Form107,Status,Develop Product QE (DPQE)
Project ID,,,,,,,,,
2854,Product - GDC 7500,2013-04-30,NaT,2019-07-01,2018-10-12,Global,PDP,Development,"Kuo, Chris"
5711,Product - GLA 85,2014-10-30,NaT,2018-12-07,2016-08-31,Global,PDP,Development,"Kuo, Chris"
7563,Mercury,NaT,NaT,2022-01-17,2021-04-23,Global,PDP,Development,"Fu, Ernest"
7899,Product - GI 275,2018-04-04,NaT,2019-04-02,2019-03-22,Global,PDP,Development,"Kuo, Chris"
8510,Xero L60i Laser Navigator,2022-10-19,2023-07-17,2024-01-08,2023-10-16,Global,PDP,Development,"Kuo, Chris"


In [211]:
value_vars = ['Working Prototype Built and Tested','Final Factory Prototypes Built','PR Date','MP Date']
pmsystem_project_dpqe_melted = df_consumer_dpqe.reset_index().melt(id_vars= ['Project ID','Project Name','Develop  Product QE (DPQE)'],value_vars=value_vars)
pmsystem_project_dpqe_melted.head()

,Project ID,Project Name,Develop Product QE (DPQE),variable,value
0,2854,Product - GDC 7500,"Kuo, Chris",Working Prototype Built and Tested,2013-04-30
1,5711,Product - GLA 85,"Kuo, Chris",Working Prototype Built and Tested,2014-10-30
2,7563,Mercury,"Fu, Ernest",Working Prototype Built and Tested,NaT
3,7899,Product - GI 275,"Kuo, Chris",Working Prototype Built and Tested,2018-04-04
4,8510,Xero L60i Laser Navigator,"Kuo, Chris",Working Prototype Built and Tested,2022-10-19


In [212]:
pmsystem_project_dpqe_melted.rename(columns={'Develop  Product QE (DPQE)':'DPQE','variable':'Phase','value':'Date'},inplace=True)

In [213]:
pmsystem_project_dpqe_melted.Date = pmsystem_project_dpqe_melted.Date.dt.strftime('%Y-%m')

In [214]:
pmsystem_project_dpqe_melted = pmsystem_project_dpqe_melted[(pmsystem_project_dpqe_melted.Date.isnull()) | (pmsystem_project_dpqe_melted.Date>=start_from)]

In [215]:
pmsystem_project_dpqe_melted.isnull().sum()

Project ID        0
Project Name      0
DPQE              0
Phase             0
Date            125
dtype: int64

Weight socre

In [216]:
pmsystem_project_dpqe_melted.loc[:,'weighted_score'] = pmsystem_project_dpqe_melted.loc[:,'Phase'].apply(lambda phase : 1 if phase == 'Working Prototype Built and Tested' else 2 if phase=='Final Factory Prototypes Built' else 3 if phase=='PR Date' else 4 if phase == 'MP Date' else 0)

In [217]:
pmsystem_project_dpqe_melted = pmsystem_project_dpqe_melted.rename(columns={"Project ID":'PM Project Id','Project Name':'PM Project Name'})

In [218]:
pmsystem_project_dpqe_melted.head()

,PM Project Id,PM Project Name,DPQE,Phase,Date,weighted_score
2,7563,Mercury,"Fu, Ernest",Working Prototype Built and Tested,NaN,1
5,8752,Marine inReach Antenna,"Hsu, Jason(PQE)",Working Prototype Built and Tested,NaN,1
11,10617,Force Kayak Propulsion System,"Hsu, Jason(PQE)",Working Prototype Built and Tested,2023-12,1
12,10692,SaMD Sleep Apnea Risk Assessment,"Fu, Ernest",Working Prototype Built and Tested,NaN,1
15,11252,MARQ Carbon (Gen 2),"Jen, Howard",Working Prototype Built and Tested,NaN,1


# PRweb

Read

In [219]:
prweb = pd.read_excel(prweb_file, header=1, usecols=[0,3])
prweb = prweb[prweb['Project Name'] != 'GPSMAP 9000_Black Box'] # 2022/09/05, GPSMAP 9500 replace GPSMAP 9000_Black Box
prweb = prweb.query("~`Project Name`.str.contains('BMW')")
prweb.head()

,Project Name,PM Project Id
0,Force Kayak Propulsion System,10617
1,Rally 110/210 Family,16349
2,GPSMAP H1/H1i,16137
3,Engine Cut-Off System (ECOS/MOB),16198
4,Xero B60i Laser Navigation Binoculars,13229


Merge with data from PMsystem

In [220]:
prweb_pmsystem = pd.merge(left=pmsystem_project_dpqe_melted, right=prweb, how='left', left_on='PM Project Id', right_on='PM Project Id', suffixes=['_PMsystem','_prweb'])

In [221]:
prweb_pmsystem.head()

,PM Project Id,PM Project Name,DPQE,Phase,Date,weighted_score,Project Name
0,7563,Mercury,"Fu, Ernest",Working Prototype Built and Tested,NaN,1,NaN
1,8752,Marine inReach Antenna,"Hsu, Jason(PQE)",Working Prototype Built and Tested,NaN,1,NaN
2,10617,Force Kayak Propulsion System,"Hsu, Jason(PQE)",Working Prototype Built and Tested,2023-12,1,Force Kayak Propulsion System
3,10692,SaMD Sleep Apnea Risk Assessment,"Fu, Ernest",Working Prototype Built and Tested,NaN,1,NaN
4,11252,MARQ Carbon (Gen 2),"Jen, Howard",Working Prototype Built and Tested,NaN,1,MARQ Carbon


In [222]:
Blacklist = 'Mercury|SaMD Sleep Apnea Risk Assessment|GDC 7500|Connect IQ 3.x|GI 275|GLA 85|Tacx NEO Bike Plus|Tacx NEO 3M|Edge 540 840'

In [223]:
prweb_pmsystem = (prweb_pmsystem
 .query("~`PM Project Name`.str.contains(@Blacklist)")
 )


Handle GPSMAP 9000 19"

In [224]:
'Hsu, Jason(PQE);\nWang, Evelyn'.replace('\n','').split(';')

['Hsu, Jason(PQE)', 'Wang, Evelyn']

In [225]:
# GPSMAP 9000 - 19" to Wang, Evelyn, others to Hsu, Jason(PQE)
def gpsmap_9000 (project_name, DPQE):
    if project_name == 'GPSMAP 9000 - 19"':
        return DPQE.replace('\n','').split(';')[1] #Wang, Evelyn
    elif project_name in ['GPSMAP 9000 - 22"','GPSMAP 9000 - 24"','GPSMAP 9000 -27"','GPSMAP 9000_Black Box']:
        return DPQE.replace('\n','').split(';')[0] #Hsu, Jason(PQE)
    else:
        return DPQE


In [226]:
prweb_pmsystem['DPQE'] = np.vectorize(gpsmap_9000)(prweb_pmsystem['Project Name'],prweb_pmsystem['DPQE'])

In [227]:
prweb_pmsystem.drop_duplicates(inplace=True)

In [228]:
#Validate
prweb_pmsystem[prweb_pmsystem['PM Project Name'] == 'GPSMAP 9000']

,PM Project Id,PM Project Name,DPQE,Phase,Date,weighted_score,Project Name
158,9946,GPSMAP 9000,"Wang, Evelyn",PR Date,2023-06,3,"GPSMAP 9000 - 19"""
159,9946,GPSMAP 9000,"Hsu, Jason(PQE)",PR Date,2023-06,3,"GPSMAP 9000 - 24"""
160,9946,GPSMAP 9000,"Hsu, Jason(PQE)",PR Date,2023-06,3,"GPSMAP 9000 -27"""
161,9946,GPSMAP 9000,"Hsu, Jason(PQE)",PR Date,2023-06,3,"GPSMAP 9000 - 22"""
263,9946,GPSMAP 9000,"Wang, Evelyn",MP Date,2023-08,4,"GPSMAP 9000 - 19"""
264,9946,GPSMAP 9000,"Hsu, Jason(PQE)",MP Date,2023-08,4,"GPSMAP 9000 - 24"""
265,9946,GPSMAP 9000,"Hsu, Jason(PQE)",MP Date,2023-08,4,"GPSMAP 9000 -27"""
266,9946,GPSMAP 9000,"Hsu, Jason(PQE)",MP Date,2023-08,4,"GPSMAP 9000 - 22"""


Fill PM system projects' name to PRweb's if PRweb's projects name is Nan (2022/11/11 Ray's request)

In [229]:
def noPRweb(PMProjectName,ProjectName):
    if ProjectName is np.nan:
        return f"1_NoPRweb_{PMProjectName}"
    else:
        return ProjectName

In [230]:
prweb_pmsystem['Project Name'] = np.vectorize(noPRweb)(prweb_pmsystem['PM Project Name'],prweb_pmsystem['Project Name'])

In [231]:
prweb_pmsystem.Date.fillna('1_NoDate(count)', inplace=True)

In [232]:
def weighted_score_is_0_if_NoDate(Date, weighted_score):
    if Date == '1_NoDate(count)':
        return 1
    else:
        return weighted_score
prweb_pmsystem.weighted_score = np.vectorize(weighted_score_is_0_if_NoDate)(prweb_pmsystem.Date, prweb_pmsystem.weighted_score)

Extract MP date and merge to "detail table"

In [233]:
def pmsystem_mp_date(pmsystem_project):
    return(pmsystem_project
    .reset_index()
    .rename(columns={'Project ID':'PM Project Id'})
    .iloc[:,[0,4]]
    .rename(columns = {"MP Date":"MP_Date"})
    .assign(MP_Date = lambda df_ : df_.MP_Date.dt.date)
    )

In [234]:
prweb_pmsystem = pd.merge(prweb_pmsystem, pmsystem_mp_date(pmsystem_project), on='PM Project Id')

In [235]:
detail = (prweb_pmsystem
 .groupby(['DPQE','Project Name','Date','MP_Date'])
 .agg({'weighted_score':'sum'})
#  .agg({'weighted_score':'sum','Project Name':'count'})
 .sort_index(level=['Date','Project Name'])
#  .rename(columns=({'weighted_score':'Weighted_Project_counts','Project Name':'Project_counts'}))
 ['weighted_score']
 .sort_index(level='MP_Date')
 .unstack(level='Date',fill_value=0)
 .style.background_gradient(cmap="OrRd", axis=None)
)
detail

Detail dataframe

In [236]:
# detail = prweb_pmsystem.groupby(['DPQE','Project Name','Date']).agg({'weighted_score':'sum','Project Name':'count'}).sort_index(level=['Date','Project Name']).rename(columns=({'weighted_score':'Weighted_Project_counts','Project Name':'Project_counts'})).unstack(fill_value=0).style.background_gradient(cmap="OrRd", axis=None)
# detail

# Style overview DataFrame

Overview

In [237]:
overview = prweb_pmsystem.groupby(['DPQE','Date']).count()['Project Name'].sort_index(level='Date').unstack(fill_value=0).iloc[:,1:].style.background_gradient(cmap="OrRd", axis=None)
overview

#2022/11/12 iloc[:,1:] is to avoid NoDate projects (count) dominate background_gradient

Date,2023-06,2023-07,2023-08,2023-09,2023-10,2023-11,2023-12,2024-01,2024-02,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12,2025-01,2025-02,2025-04,2025-10,2026-01
DPQE,,,,,,,,,,,,,,,,,,,,,,,,
"Chen, Leo(PQE)",0,2,0,0,1,2,1,3,0,1,2,2,2,1,0,0,0,0,1,0,0,0,0,0
"Fu, Ernest",1,0,2,1,2,0,1,0,2,1,0,0,0,1,2,0,0,0,0,2,0,0,0,1
"Hsu, Jason(PQE)",7,4,7,7,3,3,4,3,1,2,3,1,0,1,2,3,0,0,1,2,0,1,1,0
"Jen, Howard",0,1,0,4,0,3,1,6,0,0,6,1,6,0,1,0,1,1,0,0,0,0,0,0
"Kuo, Chris",3,1,3,3,8,2,2,6,1,4,3,0,3,2,2,2,0,1,0,1,0,0,0,0
"Lo, Aalto",1,1,0,1,0,0,2,2,1,3,1,0,3,0,2,0,0,0,1,0,0,0,0,0
"Wang, Cooper",0,2,0,2,2,1,0,1,1,0,0,0,0,1,0,0,1,0,1,1,1,0,0,0
"Wang, Evelyn",3,1,5,1,1,4,1,1,1,2,2,4,1,1,0,1,0,1,0,2,0,1,0,0


Overview_weighted

In [238]:
overview_weighted = prweb_pmsystem.groupby(['DPQE','Date']).sum()['weighted_score'].sort_index(level='Date').unstack(fill_value=0).iloc[:,1:].style.background_gradient(cmap="OrRd", axis=None)
overview_weighted

#2022/11/12 iloc[:,1:] is to avoid 1_NoDate(count) dominate background_gradient

C:\Users\wangcooper\AppData\Local\Temp\ipykernel_16004\1236123746.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  overview_weighted = prweb_pmsystem.groupby(['DPQE','Date']).sum()['weighted_score'].sort_index(level='Date').unstack(fill_value=0).iloc[:,1:].style.background_gradient(cmap="OrRd", axis=None)


Date,2023-06,2023-07,2023-08,2023-09,2023-10,2023-11,2023-12,2024-01,2024-02,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12,2025-01,2025-02,2025-04,2025-10,2026-01
DPQE,,,,,,,,,,,,,,,,,,,,,,,,
"Chen, Leo(PQE)",0,8,0,0,2,2,1,6,0,4,6,7,8,4,0,0,0,0,4,0,0,0,0,0
"Fu, Ernest",1,0,6,4,8,0,1,0,5,4,0,0,0,3,8,0,0,0,0,8,0,0,0,4
"Hsu, Jason(PQE)",20,10,27,17,11,6,9,8,4,6,12,2,0,3,5,10,0,0,3,7,0,4,4,0
"Jen, Howard",0,3,0,7,0,3,1,12,0,0,18,2,24,0,3,0,4,4,0,0,0,0,0,0
"Kuo, Chris",5,2,9,6,19,2,7,17,4,14,8,0,11,6,8,8,0,4,0,4,0,0,0,0
"Lo, Aalto",3,4,0,1,0,0,5,2,3,8,4,0,10,0,8,0,0,0,4,0,0,0,0,0
"Wang, Cooper",0,4,0,6,8,3,0,4,1,0,0,0,0,2,0,0,3,0,4,4,4,0,0,0
"Wang, Evelyn",7,4,17,1,1,14,2,2,3,4,6,13,4,4,0,2,0,3,0,7,0,4,0,0


# 2022/09/02 request by Ray

 Find PM system projects MP dates from NOW

In [239]:
pmsystem_project2 = pd.read_excel(pmsystem_project_list_file, usecols=[0,1,5,8,9,10,12], 
    parse_dates=[2,3,4,5], engine='openpyxl')
pmsystem_project2 = pmsystem_project2[pmsystem_project2['Global/Asia']=='Global']
pmsystem_project2.head()

,Project ID,Project Name,MP Date,Category,Global/Asia,PDP/Form107,Status
0,16544.0,2024 Value Dash Cams,2024-03-18,Automotive,Global,PDP,Development
1,16319.0,Aviation Headset,2025-04-21,Automotive,Global,PDP,Development
2,15624.0,Catalyst Kart/Auto,2023-11-13,Automotive,Global,PDP,Development
3,16199.0,DC-2023 Premium Dash Cameras,2023-12-04,Automotive,Global,PDP,Development
4,16718.0,DEZL 2024,2024-06-10,Automotive,Global,PDP,Development


In [240]:
MPdate_from_now = pmsystem_project2[pmsystem_project2['MP Date'] > datetime.now()].sort_values(['Category','MP Date'], ascending=True).set_index('Category')
MPdate_from_now['MP Date'] = MPdate_from_now['MP Date'].dt.date
MPdate_from_now

,Project ID,Project Name,MP Date,Global/Asia,PDP/Form107,Status
Category,,,,,,
Automotive,15624.0,Catalyst Kart/Auto,2023-11-13,Global,PDP,Development
Automotive,16199.0,DC-2023 Premium Dash Cameras,2023-12-04,Global,PDP,Development
Automotive,16544.0,2024 Value Dash Cams,2024-03-18,Global,PDP,Development
Automotive,16539.0,Motorcycle Radar 2023,2024-04-29,Global,PDP,Development
Automotive,16249.0,dezl headset (2nd Gen),2024-05-06,Global,PDP,Development
...,...,...,...,...,...,...
Outdoor,16736.0,Approach G82,2024-12-02,Global,PDP,Development
Outdoor,17029.0,Approach S50,2025-01-01,Global,PDP,Concepts
Outdoor,16673.0,Approach Z84,2025-01-20,Global,PDP,Concepts


In [241]:
MPdate_from_now_count = MPdate_from_now.groupby('Category')['Project Name'].count().to_frame().rename({'Project Name':'Count'},axis=1)
MPdate_from_now_count

,Count
Category,
Automotive,9
Fitness,14
Garmin Labs,1
Marine,21
OEM Auto,1
Outdoor,34


In [242]:
No_MPdate = pmsystem_project2[pmsystem_project2['MP Date'].isnull()].sort_values(['Category','MP Date'], ascending=True).set_index('Category')
No_MPdate = No_MPdate[~No_MPdate.index.isin(['Aviation','Garmin Labs'])]
No_MPdate

,Project ID,Project Name,MP Date,Global/Asia,PDP/Form107,Status
Category,,,,,,
Automotive,17038.0,Dash Cam Lineup (Gen 7),NaT,Global,PDP,Concepts
Automotive,16717.0,DriveSmart 67/77/87,NaT,Global,PDP,Concepts
Automotive,16949.0,Garmin zumo XS,NaT,Global,PDP,Concepts
Automotive,17176.0,"Tread 8"" Overland Edition (entry level)",NaT,Global,PDP,Concepts
Fitness,17053.0,Bounce 2,NaT,Global,PDP,Concepts
...,...,...,...,...,...,...
Outdoor,16212.0,Marq (Gen 3),NaT,Global,PDP,Concepts
Outdoor,17061.0,MicroLED Adventure Watch (51mm),NaT,Global,PDP,Concepts
Outdoor,17099.0,Rino 8xx,NaT,Global,PDP,Concepts


In [243]:
No_MPdate_count = No_MPdate.groupby('Category')['Project Name'].count().to_frame().rename({'Project Name':'Count'},axis=1)
No_MPdate_count

,Count
Category,
Automotive,4
Fitness,9
Marine,13
OEM Auto,21
Outdoor,25


Find Avation projects MP dates from PRweb

In [244]:
prweb2 = pd.read_excel(prweb_file, header=1, usecols=[0,1,5,6])
Aviation_PRweb = prweb2[prweb2['Category'] == 'Aviation'].sort_values('MP Date')
Aviation_PRweb

,Project Name,Project Id,Category,MP Date
78,Product - Flight Stream Redesign,977,Aviation,2023-05-15
36,Large Format TXi,1155,Aviation,2024-01-01
37,Product - G3X/G500TXi Hybrid,1050,Aviation,2024-01-01
38,Product - GDU77XW,1048,Aviation,2024-01-01
35,Product - GDU147XW,1049,Aviation,2024-01-03
129,"Product - 1.3\"" GTR/GNC",984,Aviation,NaN


# to_excel

In [245]:
with pd.ExcelWriter(getcwd()+f"\GMLA_Loading_{datetime.today().strftime('%Y-%m-%d')}.xlsx", engine='openpyxl') as writer:

    overview.to_excel(writer, sheet_name='overview')

    overview_weighted.to_excel(writer, sheet_name='overview_weighted')
    
    detail.to_excel(writer, sheet_name='detail')
    #auto_adjust_xlsx_column_width(overall, writer, sheet_name="overall", margin=5)
    
    # MPdate_from_now_count.to_excel(writer, sheet_name='MPdate_from_now_count')
    # auto_adjust_xlsx_column_width(MPdate_from_now_count, writer, sheet_name="MPdate_from_now_count", margin=3)
    
    MPdate_from_now.to_excel(writer, sheet_name='MPdate_from_now')
    auto_adjust_xlsx_column_width(MPdate_from_now, writer, sheet_name="MPdate_from_now", margin=3)
    
    No_MPdate_count.to_excel(writer, sheet_name='No_MPdate_count')
    auto_adjust_xlsx_column_width(No_MPdate_count, writer, sheet_name="No_MPdate_count", margin=3)
    
    No_MPdate.to_excel(writer, sheet_name='No_MPdate')
    auto_adjust_xlsx_column_width(No_MPdate, writer, sheet_name="No_MPdate", margin=3)
    
    pmsystem_project_wo_any_date.to_excel(writer, sheet_name='pmsystem_project_wo_any_date')
    auto_adjust_xlsx_column_width(pmsystem_project_wo_any_date, writer, sheet_name="pmsystem_project_wo_any_date", margin=5)
    
    ww_asia_diff_DPQE.to_excel(writer, sheet_name='ww_asia_diff_DPQE')
    auto_adjust_xlsx_column_width(ww_asia_diff_DPQE, writer, sheet_name="ww_asia_diff_DPQE", margin=5)
    
    projects_wo_DPQE.to_excel(writer, sheet_name='projects_wo_DPQE')
    auto_adjust_xlsx_column_width(projects_wo_DPQE, writer, sheet_name="projects_wo_DPQE", margin=5)
    
    multi_dpqe.to_excel(writer, sheet_name='multi_dpqe')
    auto_adjust_xlsx_column_width(multi_dpqe, writer, sheet_name="multi_dpqe", margin=5)


    Aviation_PRweb.to_excel(writer, sheet_name='Aviation_PRweb')
    auto_adjust_xlsx_column_width(Aviation_PRweb, writer, sheet_name="Aviation_PRweb", margin=3)

